In [0]:
a# -------------------------------------------------------------------------
# CELL 1: SETUP CONNECTION
# -------------------------------------------------------------------------
# 1. Enter your Storage Account Name (Check Azure Portal if unsure)
storage_name = "stfoodsafetysk" 

# 2. Enter your Key (Go to Azure Portal -> Storage -> Access Keys -> Key 1)
storage_key = "KEY_REMOVED_FOR_SECURITY" 

# 3. Configure the connection
spark.conf.set(
    f"fs.azure.account.key.{storage_name}.dfs.core.windows.net",
    storage_key
)

print("Connection Configured.")

Connection Configured.


In [0]:
# -------------------------------------------------------------------------
# CELL 2: PROCESS CHICAGO
# -------------------------------------------------------------------------
from pyspark.sql.functions import col, lit, to_date

# 1. Read Bronze Data
df_chicago = spark.read.format("json") \
    .option("recursiveFileLookup", "true") \
    .load(f"abfss://datalake@{storage_name}.dfs.core.windows.net/bronze/chicago")

# 2. Transform & Standardize Columns
df_chicago_clean = df_chicago.select(
    col("dba_name").alias("restaurant_name"),
    col("aka_name"),
    col("license_").alias("license_number"),
    col("facility_type"),
    col("risk"),
    col("address"),
    col("city"),
    col("state"),
    col("zip").alias("zip_code"),
    to_date(col("inspection_date")).alias("inspection_date"),
    col("results").alias("result"),
    col("violations"),
    col("latitude"),
    col("longitude")
).withColumn("source_city", lit("Chicago"))

print("Chicago Data Processed.")
display(df_chicago_clean.limit(5))

Chicago Data Processed.


restaurant_name,aka_name,license_number,facility_type,risk,address,city,state,zip_code,inspection_date,result,violations,latitude,longitude,source_city
BURGER KING #1308,BURGER KING #1308,2368739,Restaurant,Risk 2 (Medium),110 E 95TH ST,CHICAGO,IL,60619,2025-12-08,Pass,"49. NON-FOOD/FOOD CONTACT SURFACES CLEAN - Comments: OBSERVED DEBRIS ON INTERIOR AND EXTERIOR SURFACES OF HOLDING UNITS, PREP TABLES, SHELVING UNITS, CABINETS. INSTRUCTED MANAGER TO CLEAN AND MAINTAIN. | 55. PHYSICAL FACILITIES INSTALLED, MAINTAINED & CLEAN - Comments: OBSERVED DEBRIS ON FLOORS UNDER ALL EQUIPMENT, ALONG WALLS AND IN ALL CORNERS IN PREP AND STORAGE AREAS. INSTRUCTED MANAGER TO CLEAN AND MAINTAIN.",41.721901009676586,-87.62006368939045,Chicago
MANNY'S BAKERY INC,MANNY'S BAKERY,2803598,Bakery,Risk 2 (Medium),3170 S ASHLAND AVE,CHICAGO,IL,60608,2025-12-05,Pass,"40. PERSONAL CLEANLINESS - Comments: FOOD HANDLERS NOT WEARING HAIR RESTRAINTS. INSTRUCTED TO WEAR HAIR RESTRAINTS WHEN HANDLING FOOD AND IN THE FOOD PREP AREA. MAINTAIN SAME. | 49. NON-FOOD/FOOD CONTACT SURFACES CLEAN - Comments: FOOD DEBRIS ON THE STORAGE RACKS INSIDE THE WALK IN COOLER. INSTRUCTED TO CLEAN AND MAINTAIN STORAGE RACKS. | 55. PHYSICAL FACILITIES INSTALLED, MAINTAINED & CLEAN - Comments: WATER STAINED CEILING TILES IN THE TOILET ROOM AND REAR PREP AREA. INSTRUCTED TO REPLACE CEILING TILES AND MAINTAIN. | 55. PHYSICAL FACILITIES INSTALLED, MAINTAINED & CLEAN - Comments: FLOORS IN THE CORNERS AND UNDER/AROUND THE HEAVY EQUIPMENT WITH DIRT, FOOD AND DEBRIS. INSTRUCTED TO CLEAN AND MAINTAIN FLOORS. | 57. ALL FOOD EMPLOYEES HAVE FOOD HANDLER TRAINING - Comments: FOOD HANDLER ON SITE WITH NO PROOF THAT HE TOOK THE FOOD HANDLER'S COURSE. INSTRUCTED TO OBTAIN AND MAINTAIN A VALID FOOD HANDLER'S CERTIFICATE.",41.835749059464845,-87.6658416717942,Chicago
LITTLE ANGELS FAMILY DAYCARE,LITTLE ANGELS FAMILY DAYCARE,2215839,Daycare Above and Under 2 Years,Risk 1 (High),6701 S EMERALD AVE,CHICAGO,IL,60621,2025-12-05,Pass,null,41.77240524117657,-87.64310796196668,Chicago
CARNICERIA EL PORVENIR,CARNICERIA EL PORVENIR,1768971,Grocery Store,Risk 1 (High),150 E KENSINGTON AVE,CHICAGO,IL,60628,2025-12-05,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOWLEDGE, AND PERFORMS DUTIES - Comments: 2-101.11 OBSERVED THE PERSON IN CHARGE WITHOUT A CITY OF CHICAGO FOOD SERVICE SANITATION MANAGERS CERTIFICATE. INSTRUCTED ALL PERSONS IN CHARGE MUST OBTAIN A CITY OF CHICAGO FOOD SERVICE SANITATION MANAGERS CERTIFICATE. PRIORITY FOUNDATION. CITATION ISSUED UNDER VIOLATION #2. 7-38-012. | 2. CITY OF CHICAGO FOOD SERVICE SANITATION CERTIFICATE - Comments: 2-101.11 OBSERVED NO CITY OF CHICAGO FOOD SERVICE SANITATION MANAGER ON SITE WHILE TCS FOODS (BEEF, DELI MEAT) ARE BEING PREPARED HANDLED AND SERVED TO THE PUBLIC. INSTRUCTED A CITY OF CHICAGO FOOD SERVICE SANITATION MANAGER MUST BE ON SITE AT ALL TIMES THE FACILITY IS OPEN AND OPERATING. PRIORITY FOUNDATION. CITATION ISSUED 7-38-012. | 5. PROCEDURES FOR RESPONDING TO VOMITING AND DIARRHEAL EVENTS - Comments: 2-501.11 OBSERVED NO WRITTEN PROCEDURE NOR ADEQUATE SUPPLIES ON SITE TO PROPERLY RESPOND TO VOMIT AND DIARRHEA EVENTS THAT CAN POTENTIALLY OCCUR INSIDE OF THE FACILITY. INSTRUCTED TO PROVIDE AT ALL TIMES. PRIORITY FOUNDATION. CITATION ISSUED 7-38-005. | 10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLIED AND ACCESSIBLE - Comments: 6-301.11 OBSERVED NO HAND WASHING CLEANSER AT THE HAND WASHING SINK LOCATED IN THE TOILET ROOM. INSTRUCTED TO PROVIDE HAND WASHING CLEANSER AT ALL HAND WASHING SINKS AT ALL TIMES. PRIORITY FOUNDATION. CITATION ISSUED 7-38-030(C). | 16. FOOD-CONTACT SURFACES: CLEANED & SANITIZED - Comments: 4-602.11 OBSERVED DEEP GROOVES ON THE CUTTING BOARDS IN THE BUTCHER PREP AREA. INSTRUCTED TO REMOVE GROOVES OR REPLACE CUTTING BOARD. OBSERVED DELI MEAT SLICER IN NEED OF DETAIL CLEANING TO REMOVE FOOD DEBRIS. INSTRUCTED TO CLEAN AND SANITIZE. | 36. THERMOMETERS PROVIDED & ACCURATE - Comments: 4-302.12 OBSERVED NO METAL STEM THERMOMETER ON SITE

In [0]:
# -------------------------------------------------------------------------
# CELL 3: PROCESS DALLAS
# -------------------------------------------------------------------------
from pyspark.sql.functions import concat_ws, coalesce

# 1. Read Bronze Data
df_dallas = spark.read.format("json") \
    .option("recursiveFileLookup", "true") \
    .load(f"abfss://datalake@{storage_name}.dfs.core.windows.net/bronze/dallas")

# 2. Find all violation columns (violation1_text, violation2_text, etc.)
violation_cols = [c for c in df_dallas.columns if "violation" in c and "text" in c]

# 3. Transform & Standardize
df_dallas_clean = df_dallas.select(
    col("program_identifier").alias("restaurant_name"),
    lit(None).alias("aka_name"),
    lit(None).alias("license_number"),
    lit("Restaurant").alias("facility_type"),
    lit(None).alias("risk"),
    col("site_address").alias("address"),
    lit("Dallas").alias("city"),
    lit("TX").alias("state"),
    col("zip").alias("zip_code"),
    to_date(col("insp_date")).alias("inspection_date"),
    lit("Pass").alias("result"), # Placeholder
    # Combine all Dallas violation columns into one text block to match Chicago
    concat_ws(" | ", *[coalesce(col(c), lit("")) for c in violation_cols]).alias("violations"),
    col("lat_long.latitude").alias("latitude"),
    col("lat_long.longitude").alias("longitude")
).withColumn("source_city", lit("Dallas"))

print("Dallas Data Processed.")
display(df_dallas_clean.limit(5))

Dallas Data Processed.


restaurant_name aka_name license_number facility_type risk address city state zip_code inspection_date result violations latitude longitude source_city TIENDA Y RESTAURANTE LA CAMPIñA SALvadorena null null Restaurant null 10818 DENNIS RD 102 Dallas TX 75229 2016-10-03 Pass 228.34 Management and Personnel. Duties. The person in charge shall ensure that: (8) employees are using proper methods to rapidly cool TCS foods that are not held hot or are not for consumption within four hours, through daily oversight of the employees' routine monitoring of food temperatures during cooling; | 228.75 Food. Time and temperature control. (e) Cooling methods. (1) Cooling shall be accomplished in accordance with the time and temperature criteria specified in subsection (d) of this section by using one or more of the following methods based on the type of food being cooled: (E) using containers that facilitate heat transfer; | 228.75 Food. Time and temperature control. (f) Time/temperature controlled for safety food, hot and cold holding. (1) Except during preparation, cooking, or cooling, or when time is used as the public health control as specified in subsection (i) of this section, and except as specified in paragraphs (2) and (3) of this subsection, TCS food shall be maintained: (B) at 5 degrees Celsius (41 degrees Fahrenheit) or less; | 228.42 Management and Personnel. Food Contamination Prevention (a) Eating, drinking, or using tobacco. (1) except as specified in paragraph (2) of this subsection, an employee shall eat, drink, or use any form of tobacco only in designated areas where the contamination of exposed food; clean equipment, utensils, and linens; unwrapped single-service and single-use articles; or other items needing protection cannot result. | 228.69 Food. Preventing contamination from the premises. (a) Food Storage. (1) Except as specified in paragraphs (2) and (3) of this subsection, food shall be protected from contamination by storing the food: (C) at least 15 centimeters (6 inches) above the floor. | 228.186 Physical Facilities. Premises, buildings, systems, rooms, fixtures, equipment, devices, and materials. (k) Controlling pests. The presence of insects, rodents, and other pests shall be controlled to eliminate their presence on the premises by: (3) using methods, if pests are found, such as trapping devices or other means of pest control as specified in õ228.204(b) and õ228.208(b) and (c) of this title; and | | | | | 228.75 Food. Time and temperature control. (d) Cooling. (1) Cooked temperature/time controlled for safety food shall be cooled: (A) within two hours, from 57 degrees Celsius (135 degrees Fahrenheit) to 21 degrees C (70 degrees Fahrenheit); and | | | | | | | 228.111 Equipment, Utensils, and Linens. Equipment, maintenance and operation. (n) Manual and mechanical warewashing equipment, chemical sanitization - temperature, pH, concentration, and hardness. A chemical sanitizer used in a sanitizing solution for a manual or mechanical operation at contact times specified in õ228.118(3) of this title shall meet the criteria in õ228.206(a) of this title, shall be used in accordance with the EPA-approved manufacturer's label use instructions, and shall be used as follows: (1) a chlorine solution shall have a minimum temperature based on the concentration and pH of the solution as listed in the following chart: Figure: 25 TAC õ228.111(n)(1) | 228.149 Water, Plumbing, and Waste. Plumbing, operation and maintenance. (a) Using a handwashing facility. (1) A handwashing facility shall be maintained so that it is accessible at all times for employee use. | Sec. 17-2.2(c)(1)(D) (c) Registered food service managers. (1) Registered food service managers required. (D) A food establishment shall have one registered food service manager employed and present in the establishment during all hours of operation, except that a registered food service manager serving multiple food establishments as authorized by Section 17-2.2(c)(1)(C

In [0]:
# -------------------------------------------------------------------------
# CELL 4: UNION AND SAVE
# -------------------------------------------------------------------------
# 1. Stack the tables (Union)
df_unified = df_chicago_clean.unionByName(df_dallas_clean, allowMissingColumns=True)

# 2. Define the Silver path
silver_path = f"abfss://datalake@{storage_name}.dfs.core.windows.net/silver/unified_inspections"

# 3. Write to Azure Data Lake (Parquet format)
df_unified.write \
    .format("parquet") \
    .mode("overwrite") \
    .save(silver_path)

print(f"SUCCESS! Data saved to: {silver_path}")

SUCCESS! Data saved to: abfss://datalake@stfoodsafetysk.dfs.core.windows.net/silver/unified_inspections
